In [184]:
import ast
import requests
from datetime import date, timedelta
list = []
class Krx:

    isin_code = 'KR7005930003'
    start_date = '2019/04/20'
    end_date = '2019/06/20'
    


    def main(self):

        start = date(int(self.start_date.split("/")[0]), int(self.start_date.split("/")[1]), int(self.start_date.split("/")[2]))
        end = date(int(self.end_date.split("/")[0]), int(self.end_date.split("/")[1]), int(self.end_date.split("/")[2]))

        delta = end - start

        day_price_data = self.get_day_price()
        short_stock_selling_data = self.get_short_stock_selling()

        for day in range(delta.days+1):
            d = start + timedelta(days=day)
            key = str(d).replace("-", "")
            if key in day_price_data:
                list.append([str(d).replace("-", "/"), day_price_data[key][0], day_price_data[key][1], day_price_data[key][2], 
                             day_price_data[key][3], day_price_data[key][4], short_stock_selling_data[key][2], short_stock_selling_data[key][3]])
       

    def get_day_price(self):

        otp = requests.get('http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?bld=MKD/04/0402/04020100/mkd04020100t3_02&name=chart')

        parameters = {
            'isu_cd': self.isin_code,
            'fromdate': self.start_date.replace("/", ""),
            'todate': self.end_date.replace("/", ""),
            'pagePath': '/contents/MKD/04/0402/04020100/MKD04020100T3T2.jsp',
            'code': otp.content
        }

        res = requests.post('http://marketdata.krx.co.kr/contents/MKD/99/MKD99000001.jspx', parameters)

        data = ast.literal_eval(res.text)['block1']

        result = {}

        for item in data:
            # tdd_clsprc : 종가
            # acc_trdval : 거래대금
            # tdd_opnprc : 시가
            # tdd_hgprc : 고가
            # tdd_lwprc : 저가
            result[item['trd_dd'].replace("/", "")] = (item['tdd_clsprc'], item['tdd_opnprc'], item['tdd_hgprc'], item['tdd_lwprc'], item['acc_trdval'])

        return result

    def get_short_stock_selling(self):

        # reverse engineered from the source at here https://finance.naver.com/item/short_trade.nhn?code=005930
        otp = requests.get('https://short.krx.co.kr/contents/COM/GenerateOTP.jspx?bld=SRT/02/02010100/srt02010100X&name=form')

        parameters = {
            'isu_cd': self.isin_code,
            'strt_dd': self.start_date.replace("/", ""),
            'end_dd': self.end_date.replace("/", ""),
            'pagePath': '/contents/SRT/02/02010100/SRT02010100X.jsp',
            'code': otp.content
        }

        res = requests.post('https://short.krx.co.kr/contents/SRT/99/SRT99000001.jspx', parameters)

        data = ast.literal_eval(res.text)['block1']

        result = {}

        for item in data:
            # cvsrtsell_trdvol : 공매도 거래량
            # str_const_val1 : 공매도 잔고량
            # cvsrtsell_trdval : 공매도 거래대금
            # str_const_val2 : 공매도 잔고금액
            result[item['trd_dd'].replace("/", "")] = (item['cvsrtsell_trdvol'], item['str_const_val1'], item['cvsrtsell_trdval'], item['str_const_val2'])

        return result


if __name__ == "__main__":
    Krx().main()


In [185]:
list

[['2019/04/22',
  '45,350',
  '45,400',
  '45,900',
  '45,100',
  '273,125,405,900',
  '4,274,865,200',
  '228,801,543,300'],
 ['2019/04/23',
  '45,200',
  '45,050',
  '45,500',
  '45,000',
  '313,179,774,125',
  '26,153,015,200',
  '235,840,808,400'],
 ['2019/04/24',
  '44,750',
  '45,400',
  '45,650',
  '44,150',
  '594,100,467,458',
  '26,061,974,400',
  '239,252,742,500'],
 ['2019/04/25',
  '44,650',
  '44,250',
  '45,000',
  '44,100',
  '485,378,095,190',
  '12,385,413,750',
  '233,646,261,350'],
 ['2019/04/26',
  '44,850',
  '44,200',
  '45,000',
  '43,800',
  '434,008,341,453',
  '13,765,144,250',
  '231,788,029,200'],
 ['2019/04/29',
  '46,150',
  '45,150',
  '46,150',
  '45,100',
  '401,124,009,550',
  '11,859,479,200',
  '222,374,098,050'],
 ['2019/04/30',
  '45,850',
  '46,000',
  '46,300',
  '45,350',
  '509,855,767,200',
  '14,499,690,750',
  '202,960,847,950'],
 ['2019/05/02',
  '45,900',
  '45,500',
  '46,150',
  '45,400',
  '395,453,520,550',
  '6,465,088,700',
  '218,3

In [186]:
import pandas as pd
df = pd.DataFrame(list, columns=["년/월/일", "종가", "시가", "고가", "저가", "거래대금", "공매도거래대금", "공매도잔고금액"])

# REMOVE COMMA
columns_values = []
columns_values.append(df.columns.values)
for i in range(len(columns_values[0])):
    df[columns_values[0][i]] = df[columns_values[0][i]].str.replace(',','')
df.head()

,년/월/일,종가,시가,고가,저가,거래대금,공매도거래대금,공매도잔고금액
0,2019/04/22,45350,45400,45900,45100,273125405900,4274865200,228801543300
1,2019/04/23,45200,45050,45500,45000,313179774125,26153015200,235840808400
2,2019/04/24,44750,45400,45650,44150,594100467458,26061974400,239252742500
3,2019/04/25,44650,44250,45000,44100,485378095190,12385413750,233646261350
4,2019/04/26,44850,44200,45000,43800,434008341453,13765144250,231788029200


In [187]:
# 전일대비등락률 (종가, 거래대금, 공매도, 시가/종가)
Close = ['']
Volumes = ['']
ShortSelling = ['']
Open_Close = ['']

for k in range(1, len(df)-1): #처음 가격은 계산 안 되므로 1, 마지막 가격에는 값이 없는 row가 있으므로 -1
    Close.append(int(df['종가'][k]) / int(df['종가'][k-1]))
    Volumes.append(int(df['거래대금'][k]) / int(df['거래대금'][k-1]))
    ShortSelling.append(int(df['공매도잔고금액'][k]) / int(df['공매도잔고금액'][k-1]))
    Open_Close.append(int(df['시가'][k+1]) / int(df['종가'][k])) # 다음날 시가갭이면 1보다 큼
Close.append('')
Volumes.append('')
ShortSelling.append('')
Open_Close.append('')
# Column 삽입
# https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/
df.insert(8, "Close", Close)
df.insert(9, "Volumes", Volumes)
df.insert(10, "ShortSelling", ShortSelling)
df.insert(11, "Open_Close", Open_Close)
df.head()

# 날짜 헷갈리면 안 됨 feature들은 전일대비당일가격, Open_Close는 당일대비다음날시가(label이기 때문이다.)

,년/월/일,종가,시가,고가,저가,거래대금,공매도거래대금,공매도잔고금액,Close,Volumes,ShortSelling,Open_Close
0,2019/04/22,45350,45400,45900,45100,273125405900,4274865200,228801543300,,,,
1,2019/04/23,45200,45050,45500,45000,313179774125,26153015200,235840808400,0.996692,1.14665,1.03077,1.00442
2,2019/04/24,44750,45400,45650,44150,594100467458,26061974400,239252742500,0.990044,1.897,1.01447,0.988827
3,2019/04/25,44650,44250,45000,44100,485378095190,12385413750,233646261350,0.997765,0.816997,0.976567,0.989922
4,2019/04/26,44850,44200,45000,43800,434008341453,13765144250,231788029200,1.00448,0.894165,0.992047,1.00669


In [188]:
# row 첫번째와 마지막 삭제
# https://thispointer.com/python-pandas-how-to-drop-rows-in-dataframe-by-index-labels/
df = df.drop([df.index[0], df.index[-1]])
df.head()

,년/월/일,종가,시가,고가,저가,거래대금,공매도거래대금,공매도잔고금액,Close,Volumes,ShortSelling,Open_Close
1,2019/04/23,45200,45050,45500,45000,313179774125,26153015200,235840808400,0.996692,1.14665,1.03077,1.00442
2,2019/04/24,44750,45400,45650,44150,594100467458,26061974400,239252742500,0.990044,1.897,1.01447,0.988827
3,2019/04/25,44650,44250,45000,44100,485378095190,12385413750,233646261350,0.997765,0.816997,0.976567,0.989922
4,2019/04/26,44850,44200,45000,43800,434008341453,13765144250,231788029200,1.00448,0.894165,0.992047,1.00669
5,2019/04/29,46150,45150,46150,45100,401124009550,11859479200,222374098050,1.02899,0.924231,0.959386,0.99675


In [189]:
# label값을 True와 False로 변환
df['Open_Close'] = df['Open_Close'] >= 1
df.head()

,년/월/일,종가,시가,고가,저가,거래대금,공매도거래대금,공매도잔고금액,Close,Volumes,ShortSelling,Open_Close
1,2019/04/23,45200,45050,45500,45000,313179774125,26153015200,235840808400,0.996692,1.14665,1.03077,True
2,2019/04/24,44750,45400,45650,44150,594100467458,26061974400,239252742500,0.990044,1.897,1.01447,False
3,2019/04/25,44650,44250,45000,44100,485378095190,12385413750,233646261350,0.997765,0.816997,0.976567,False
4,2019/04/26,44850,44200,45000,43800,434008341453,13765144250,231788029200,1.00448,0.894165,0.992047,True
5,2019/04/29,46150,45150,46150,45100,401124009550,11859479200,222374098050,1.02899,0.924231,0.959386,False


In [190]:
# LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X = df[['Close', 'Volumes', 'ShortSelling']]
y = df['Open_Close']
X_train, X_test, y_train, y_test = train_test_split(X, y)
logreg = LogisticRegression().fit(X_train, y_train)
logreg.score(X_test, y_test)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.4